In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("data/")
x_train = mnist.train.images
x_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_test = mnist.test.labels.astype("int")

print(y_train.shape)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
(55000,)


In [2]:
n_inputs = 28*28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
learning_rate = 0.01


tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=(None, n_inputs), name="x")
y = tf.placeholder(tf.int32, shape=(None), name="y")


# Create model
def neural_net(x):
    hidden1 = tf.layers.dense(x, n_hidden1,name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    return logits


logits=neural_net(x)

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()


In [3]:
for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
    print(var)

<tf.Variable 'hidden1/kernel:0' shape=(784, 300) dtype=float32_ref>
<tf.Variable 'hidden1/bias:0' shape=(300,) dtype=float32_ref>
<tf.Variable 'hidden2/kernel:0' shape=(300, 100) dtype=float32_ref>
<tf.Variable 'hidden2/bias:0' shape=(100,) dtype=float32_ref>
<tf.Variable 'outputs/kernel:0' shape=(100, 10) dtype=float32_ref>
<tf.Variable 'outputs/bias:0' shape=(10,) dtype=float32_ref>


In [4]:
def get_weights_variable(layer_name):
    with tf.variable_scope(layer_name, reuse=True):
        variable = tf.get_variable('kernel')
    return variable

In [7]:
weights_layer1 = get_weights_variable(layer_name='outputs')
weights_layer2 = get_weights_variable(layer_name='hidden2')

session = tf.Session()
session.run(init)
w1 = session.run(weights_layer1)
print(w1)

[[  3.90167236e-02   1.54658884e-01   1.98566496e-01  -1.93179309e-01
    9.61099267e-02   8.93144608e-02   1.54708058e-01  -1.83324158e-01
   -1.76736563e-02   1.12689883e-02]
 [  1.68515444e-02   1.86849415e-01  -1.62154809e-01  -1.11750551e-01
    8.94378126e-02   1.17954016e-02   5.12891114e-02   9.22330916e-02
   -4.34926152e-03   2.16087848e-01]
 [  1.14976168e-02  -3.26044410e-02  -2.19722077e-01   3.25602889e-02
   -1.20333016e-01  -5.85731268e-02  -3.74121219e-02  -1.10144101e-01
   -4.52603102e-02  -6.07701391e-02]
 [  1.12264544e-01   1.63107425e-01  -3.44285369e-02   1.31067842e-01
   -8.33428502e-02   8.19375813e-02  -1.45259947e-01  -1.98229104e-01
   -2.35104114e-02   8.10900927e-02]
 [ -2.16244653e-01   7.40278661e-02   5.72225451e-02  -1.69042796e-01
   -2.45852023e-02   5.84765673e-02  -1.75636292e-01  -2.15564549e-01
   -1.24033399e-01   1.82006717e-01]
 [  1.55341029e-02  -1.27625316e-01   9.04093683e-02  -2.11476669e-01
    8.24811757e-02  -9.92950201e-02  -6.63889

In [8]:
saver = tf.train.Saver()

n_epochs = 10
batch_size = 50

for epoch in range(n_epochs):
    for iteration in range(mnist.train.num_examples // batch_size):
        X_batch, y_batch = mnist.train.next_batch(batch_size)
        session.run(training_op, feed_dict={x: X_batch, y: y_batch})
    acc_train = session.run(accuracy,feed_dict={x: X_batch, y: y_batch})
    acc_test = session.run(accuracy,feed_dict={x: x_test, y: y_test})
    print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    #w1 = session.run(weights_layer1)
    #print(w1)

save_path = saver.save(session, "models/ann_final.ckpt")

0 Train accuracy: 0.94 Test accuracy: 0.9006
1 Train accuracy: 0.96 Test accuracy: 0.9187
2 Train accuracy: 0.92 Test accuracy: 0.9294
3 Train accuracy: 0.92 Test accuracy: 0.936
4 Train accuracy: 0.9 Test accuracy: 0.9379
5 Train accuracy: 0.98 Test accuracy: 0.9451
6 Train accuracy: 0.96 Test accuracy: 0.9471
7 Train accuracy: 0.98 Test accuracy: 0.952
8 Train accuracy: 0.94 Test accuracy: 0.9529
9 Train accuracy: 0.96 Test accuracy: 0.9571


In [9]:
w11 = session.run(weights_layer1)
print(w11.shape)

(100, 10)
